# Forest Fire Spreading 


## Background

Wildfires are one of the most harmful phenomena in Chile burning thousands of forest hectares every year destroying significant flora and fauna, and affecting the air, water cycles, and the ecosystem. Wildfires start by varied reasons, ranging from reckless human behavior to extreme weather and environmental conditions

## Main Goal

Build a mathematical model based on SPDE qualitatively realistic with the phenomenon behaviour. 

## Objectives
1. Start with an agent based model, working with the weather conditions and initial fires focus. 
2. Get a continuous model based on SPDE from the ABM.
3. Assess wind relevance to the model.

## Cellular Automata

In [1]:
import firemodels.cellularautomata as ca
import firemodels.cellularautomatalayers as cal
import firemodels.discrete.temperature as dtemp
import firemodels.continuous.temperature as ctemp
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def fireFocus(M, N, i, j, size):
    focus = np.zeros((M, N))
    focus[i-size:i+size, j-size:j+size] = np.ones((2*size, 2*size)) 
    return focus

In [ ]:
def plotStates(t=0):
    play = widgets.Play(
        value=0,
        min=0,
        max=len(states)-1,
        step=1,
        description="Press play",
        disabled=False
    )
    slider = widgets.IntSlider()
    widgets.jslink((play, 'value'), (slider, 'value'))
    widgets.HBox([play, slider])
    widgets.interact(automata.plotStates, t=play)

In [ ]:
# Testing
M = 101
N = 101
initial = fireFocus(M, N, 50, 50, 2)
rule = .35
#neighborhood = 'vonneumann'
neighborhood = 'moore'
times = 100

In [ ]:
automata = ca.new(M, N, initial, rule, neighborhood)
states = automata.propagate(times)

In [ ]:
plotStates()

# Cellular Automata with layers

In [ ]:
temperature = np.load('data/temperature100x100.npy')
wind_speed = np.load('data/wind_speed100x100.npy')
humidity = np.load('data/humidity100x100.npy')
pressure = np.load('data/pressure100x100.npy')

In [ ]:
temperature = temperature / np.max(temperature)
wind_speed = wind_speed / np.max(wind_speed)
humidity = humidity / np.max(humidity)
pressure = pressure / np.max(pressure)

In [ ]:
# Testing
(M, N) = temperature.shape
wd = np.ones_like(temperature)*0
world = [temperature, wind_speed, wd, humidity, pressure]
initial = fireFocus(M, N, 50, 50, 2)
#neighborhood = 'vonneumann'
neighborhood = 'moore'
alpha = .5
beta = 1-alpha
times = 200
rule = .3

In [ ]:
automata = cal.new(initial, world, neighborhood, rule, alpha, beta)
states = automata.propagate(times)

In [ ]:
plotStates()

* $BT$: Burning trees
* $T$: Temperature

\begin{equation}
    BT_{t+1} = BT_{t} + F(BT_t, T_t, W_t, T_t, P_T)
\end{equation}
\begin{equation}
    T_{t+1} = F_t +  G(BT_t, T_t, W_t, T_t, P_T)
\end{equation}
\begin{equation}
    W_{t+1} = W_t +  I( W_t)
\end{equation}

## Temperature

### Discrete

\begin{equation}
    \begin{split}
        T_{i,j}^{k+1} & = T_{i,j}^k - \frac45 T_{i,j}^k 
            + \frac15\left(T_{i-1,j}^k+T_{i+1,j}^k+T_{i,j-1}^k+T_{i,j+1}^k\right) \\
            & = \frac15\left(T_{i-1,j}^k+T_{i+1,j}^k+T_{i,j}^k+T_{i,j-1}^k+T_{i,j+1}^k\right)
    \end{split}
\end{equation}

In [2]:
def temperatureFocus(M, N):
    x = np.linspace(0, 1, N)
    y = np.linspace(0, 1, M)
    X, Y = np.meshgrid(x, y)
    return 1e3*np.exp(-1000*((X-.5)**2 + (Y-.5)**2))

In [ ]:
def plotTemperatures(model, t=0):
    play = widgets.Play(
        value=0,
        min=0,
        max=len(dtemps)-1,
        step=1,
        description="Press play",
        disabled=False
    )
    slider = widgets.IntSlider()
    widgets.jslink((play, 'value'), (slider, 'value'))
    widgets.HBox([play, slider])
    widgets.interact(model.plotTemperatures, t=play)

In [ ]:
M, N = 100, 100
times = 100
initial = temperatureFocus(M, N)
sigma = 0#1/30

In [ ]:
dte = dtemp.new(initial, sigma)
dtemps = dte.propagate(times)

In [ ]:
plotTemperatures(dte)

### Continuous

\begin{equation}
    \frac{\partial T}{\partial t} = \mu\Delta T
\end{equation}

In [15]:
def plotCTemperatures(temperatures, t=0):
    play = widgets.Play(
        value=0,
        min=0,
        max=len(temperatures)-1,
        step=1,
        description="Press play",
        disabled=False
    )
    slider = widgets.IntSlider()
    widgets.jslink((play, 'value'), (slider, 'value'))
    widgets.HBox([play, slider])
    widgets.interact(ct.plotTemperatures, t=play, temperatures=widgets.fixed(temperatures))

In [16]:
# The resolution have to be lower than discrete version for computation of F
M, N = 50, 50  

# Initial conditions
c_in = temperatureFocus(M, N)

# Parameters
mu = 1/5 
T = 100
dt = 1e-3

In [17]:
# We have to include border conditions, for now only 
# use dirichlet f(x,y) = u(x,y) for (x,y) \in \partial\Omega
ct = ctemp.new(c_in, mu, dt, T)
pde_temp = ct.solvePDE()

In [18]:
plotCTemperatures(pde_temp)

interactive(children=(Play(value=0, description='Press play', max=99), Output()), _dom_classes=('widget-interact',))

### Stochastic Approach 1

In [19]:
spde1_temps = ct.solveSPDE1()

In [20]:
plotCTemperatures(spde1_temps)

interactive(children=(Play(value=0, description='Press play'), Output()), _dom_classes=('widget-interact',))

In [21]:
spde2_temps = ct.solveSPDE2()

In [22]:
plotCTemperatures(spde2_temps)

interactive(children=(Play(value=0, description='Press play'), Output()), _dom_classes=('widget-interact',))